# Logistic Regression with Newton's Method

This is a response to Siraj Raval's [Coding Challenge](https://github.com/llSourcell/Second_Order_Optimization_Newtons_Method) 
In this notebook, we will be building a logistic regression model using Newton's 2nd Order Optimization Method, as well as the regular batch gradient descent.

We will be using the Kaggle [Breast Cancer Wisconsin](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data) Data Set to classify if a tumour is malignant or benign, based on 30 features, such as the mean radius.

References:  
[Logistic Regression Newton's Method](https://github.com/llSourcell/logistic_regression_newtons_method) by Siraj Raval  
[CS229 - Logistic Regression](http://cs229.stanford.edu/notes/cs229-notes1.pdf) by Andrew Ng

In [2]:
## Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Preparation
We will begin by doing some simple data cleaning and preparation before delving into the logistic regression and newton's method formulas.

In [3]:
data = pd.read_csv('data.csv')

## Dropping a unused fields
fields_to_drop = ['id', 'Unnamed: 32'] 
data = data.drop(fields_to_drop, axis=1)

## Converting diagnosis to int - 1 for malignant, 0 - for benign
d = {'M': 1, 'B': 0}
data['diagnosis'] = data['diagnosis'].map(d)

## Visualising the data set
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Splitting Data into Train and Test Sets

In [4]:
## Using 10% of dataset for testing, 10% for CV
test_split_idx = int(data.shape[0]*0.9) 
val_split_idx = int(data.shape[0]*0.8) 

test_data = data[test_split_idx:]
val_data = data[val_split_idx:test_split_idx]
data = data[:val_split_idx]

## Separating data to features and targets
train_Y, train_X = data['diagnosis'], data.drop('diagnosis', axis=1)
val_Y, val_X = val_data['diagnosis'], val_data.drop('diagnosis', axis=1)
test_Y, test_X = test_data['diagnosis'], test_data.drop('diagnosis', axis=1)

In [5]:
train_X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Logistic Regression
Logistic Regression is a method in machine learning for classification problems to output discrete values. For example, given an input of the tumour size, the logistic function can classify it as malignant (1) or benign (0).

In [6]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

## Newton's Method
Newton's method is a second-order optimization algorithm that can help us find the best weights in our logistic function in fewer iterations compared to batch gradient descent.

The generalization of Newton’s method to a multidimensional setting (also called the Newton-Raphson method) is given by:
![](images/newton.png)

Where the Hessian is represented by:
![](images/hessian.png)

For Logistic Regression, the Hessian is given by:

$$
Hf(\beta) = -X^TWX
$$
and the gradient is:

$$
\nabla f(\beta) = X^T(y-p)
$$
where
$$
W := \text{diag}\left(p(1-p)\right)
$$  
and $p$ are the predicted probabilites computed at the current value of $\beta$.


In [16]:
def newton_step(curr, y, X, reg=None):
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T  # probability matrix - N x 1
    W = np.diag((p*(1-p))[:,0]) # N by N diagonal matrix
    hessian = X.T.dot(W).dot(X)  # 30 by 30 matrix
    grad = X.T.dot(y-p)  # 30 by 1 matrix
    
    # regularization step
    if reg:
        step = np.dot(np.linalg.inv(hessian + reg*np.eye(curr.shape[0])), grad)
    else:
        step = np.dot(np.linalg.inv(hessian), grad)
        
    beta = curr + step
    
    return beta

In [35]:
def check_convergence(beta_old, beta_new, tol, iters):
    coef_change = np.abs(beta_old - beta_new)
    return not (np.any(coef_change>tol) and iters < max_iters)

In [36]:
def test_model(X, y, beta):
    prob = np.array(sigmoid(X.dot(beta)))
    
    ## Converting prob to prediction, >.5 = True, <.5 = False
    prob = np.greater(prob, 0.5*np.ones((prob.shape[1],1)))
    accuracy = np.count_nonzero(np.equal(prob, y))/prob.shape[0] * 100
    return accuracy

### Training

In [91]:
## Hyperparameters
max_iters = 20
tol=0.1 # convergence tolerance
reg_term = 1

beta_old, beta = np.ones((30,1)), np.zeros((30,1))
iter_count = 0
coefs_converged = False

while not coefs_converged:
    print('Iteration: {}'.format(iter_count))
    print('Validation Accuracy: {}%'.format(
        test_model(val_X, val_Y.to_frame(), beta_old)))
    beta_old = beta
    beta = newton_step(beta, train_Y.to_frame(), train_X, reg_term)
    iter_count += 1
    coefs_converged = check_convergence(beta_old, beta, tol, iter_count)

Iteration: 0
Validation Accuracy: 21.052631578947366%
Iteration: 1
Validation Accuracy: 78.94736842105263%
Iteration: 2
Validation Accuracy: 96.49122807017544%
Iteration: 3
Validation Accuracy: 98.24561403508771%
Iteration: 4
Validation Accuracy: 98.24561403508771%
Iteration: 5
Validation Accuracy: 92.98245614035088%
Iteration: 6
Validation Accuracy: 92.98245614035088%
Iteration: 7
Validation Accuracy: 92.98245614035088%
Iteration: 8
Validation Accuracy: 92.98245614035088%
Iteration: 9
Validation Accuracy: 92.98245614035088%
Iteration: 10
Validation Accuracy: 92.98245614035088%
Iteration: 11
Validation Accuracy: 92.98245614035088%
Iteration: 12
Validation Accuracy: 92.98245614035088%
Iteration: 13
Validation Accuracy: 92.98245614035088%
Iteration: 14
Validation Accuracy: 92.98245614035088%
Iteration: 15
Validation Accuracy: 92.98245614035088%
Iteration: 16
Validation Accuracy: 92.98245614035088%
Iteration: 17
Validation Accuracy: 92.98245614035088%
Iteration: 18
Validation Accuracy: 92

### Testing
As observed, using Newton's Method, the model converges in very few number of iterations and achieves a high test accuracy of 96%. Typically, Newton's method enjoys faster convergence than batch gradient descent, but each iteration takes long as finding and inverting the Hessian is computationally expensive.

In [92]:
print('After {} Iterations'.format(iter_count))
print('Test Accuracy: {}%'.format(
        test_model(test_X, test_Y.to_frame(), beta)))

After 20 Iterations
Test Accuracy: 96.49122807017544%


In [102]:
beta

,diagnosis
radius_mean,-1.417481
texture_mean,-2.235823
perimeter_mean,-8.463989
area_mean,-6.566313
smoothness_mean,-0.014455
compactness_mean,0.000690
concavity_mean,0.015848
concave points_mean,0.007351
symmetry_mean,-0.027443
fractal_dimension_mean,-0.010880


## Batch Gradient Ascent
To optimize the beta values of the logistic function, we can also use the following gradient ascent update rule:
![](images/sgd.png)

In [94]:
def gd_step(curr, y, X, lr=0.0000001):
    hx = X.dot(curr)
    p = np.array(sigmoid(hx))
    change = lr * (X.T.dot(y-p))
    beta = curr + change  
    
    return beta

In [99]:
# Hyperparameters
batch_size = 50
lr = 0.0001
max_iters = 51

beta_old, beta = np.ones((30,1)), np.zeros((30,1))
iter_count = 0

while iter_count < max_iters:
    if iter_count % 10 == 0:
        print('Epoch: {}'.format(iter_count))
        print('Validation Accuracy: {}%'.format(
            test_model(val_X, val_Y.to_frame(), beta)))
    beta_old = beta
    for i in range(0, train_X.shape[0], batch_size):
        beta = gd_step(beta, train_Y[i:i+batch_size].to_frame(), 
                        train_X[i:i+batch_size], lr)
    iter_count += 1

Epoch: 0
Validation Accuracy: 78.94736842105263%


/Users/jiarui1/anaconda/envs/dlp3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


Epoch: 10
Validation Accuracy: 43.859649122807014%
Epoch: 20
Validation Accuracy: 91.22807017543859%
Epoch: 30
Validation Accuracy: 87.71929824561403%
Epoch: 40
Validation Accuracy: 92.98245614035088%
Epoch: 50
Validation Accuracy: 92.98245614035088%


### Testing
While  batch gradient ascent achieves a similar accuracy of 94% after 100 iterations, a learning rate has to be determined alongside with the batch size.

In [100]:
print('After {} Iterations'.format(iter_count))
print('Test Accuracy: {}%'.format(
        test_model(test_X, test_Y.to_frame(), beta)))

After 51 Iterations
Test Accuracy: 94.73684210526315%


/Users/jiarui1/anaconda/envs/dlp3/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


In [103]:
beta

,diagnosis
radius_mean,-1.417481
texture_mean,-2.235823
perimeter_mean,-8.463989
area_mean,-6.566313
smoothness_mean,-0.014455
compactness_mean,0.000690
concavity_mean,0.015848
concave points_mean,0.007351
symmetry_mean,-0.027443
fractal_dimension_mean,-0.010880
